In [ ]:
#|hide
#|eval: false
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

5118.17s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [ ]:
#|default_exp layers

In [ ]:
#|export
# Python native modules
from copy import deepcopy
# Third party libs
from fastrl.torch_core import *
from torch import nn
import torch
from fastcore.all import L,Self,partialler
import numpy as np
# Local modules

# Layers
> Functions and Modules for RL that are pure pytorch

In [ ]:
#|export
# 3dHistogram rendering code taken from:
# https://stackoverflow.com/questions/60432713/filled-3d-histogram-from-2d-histogram-with-plotly
# and
# https://community.plotly.com/t/adding-a-shape-to-a-3d-plot/1441/10
def triangulate_histogram(x, y, z):
   
    if len(x) != len(y) != len(z) :
        raise ValueError("The  lists x, y, z, must have the same length")
    n = len(x)
    if n % 2 :
        raise ValueError("The length of lists x, y, z must be an even number") 
    pts3d = np.vstack((x, y, z)).T
    pts3dp = np.array([[x[2*k+1], y[2*k+1], 0] for k in range(1, n//2-1)])
    pts3d = np.vstack((pts3d, pts3dp))
    #triangulate the histogram bars:
    tri = [[0,1,2], [0,2,n]]
    for k, i  in zip(list(range(n, n-3+n//2)), list(range(3, n-4, 2))):
        tri.extend([[k, i, i+1], [k, i+1, k+1]])
    tri.extend([[n-3+n//2, n-3, n-2], [n-3+n//2, n-2, n-1]])      
    return pts3d, np.array(tri)

def show_sequential_layer_weights(seq:nn.Sequential,title='Layer weights'):
    import plotly.express as px
    import plotly.io as pio
    import pandas as pd
    import plotly.graph_objects as go
    pio.renderers.default = "plotly_mimetype+notebook_connected"

    weights = {}
    counter = {}
    def append_weight_dict(m):
        if type(m) == nn.Linear:
            counter['ln'] = counter.get('ln',0)+1
            weights[f"ln_{counter['ln']}"] = to_detach(m.weight.view(-1,)).numpy()
        elif type(m) == nn.Conv2d:
            counter['conv'] = counter.get('conv',0)+1
            weights[f"conv_{counter['conv']}"] = to_detach(m.weight.view(-1,)).numpy()

    seq.apply(append_weight_dict)

    max_len = max([a.shape[0] for a in weights.values()])

    for k,v in weights.items():
        pre_shape = v.shape[0]
        pad = (max_len-pre_shape)//2
        weights[k] = np.pad(v,pad)
        diff = max_len-weights[k].shape[0]
        if diff!=0: 
            weights[k] = np.hstack((weights[k],np.zeros(diff)))
            
    fig=go.Figure()
    for layer,weights in weights.items():
        a0=weights.tolist()
        a0=np.repeat(a0,2).tolist()
        a0.insert(0,0)
        a0.pop()
        a0[-1]=0
        a1=np.arange(weights.shape[0]).tolist() 
        a1=np.repeat(a1,2)
    
        verts, tri = triangulate_histogram([layer]*len(a0), a1, a0)
        x, y, z = verts.T
        I, J, K = tri.T
        z = np.round(z.astype(float),4).astype(str)
        fig.add_traces(go.Mesh3d(x=x, y=y, z=z, 
                                 i=I, j=J, k=K, opacity=0.7))

    fig.update_layout(
        scene=dict(
            xaxis_title='Layer',
            yaxis_title='Neuron',
            zaxis_title='Weight Value',
        ),
        width=700,
        height=600,
        autosize=False,
        margin=dict(l=30, r=30, b=50, t=10),
        scene_camera_eye_z=0.8,
    )
    return fig.show()

In [ ]:
torch.manual_seed(0)
layers = nn.Sequential(
    nn.Linear(2,12),
    nn.ReLU(),
    nn.Linear(12,6),
    nn.ReLU(),
    nn.Linear(6,1)
)

show_sequential_layer_weights(layers)

In [ ]:
#|export
def init_xavier_uniform_weights(m:Module,bias=0.01):
    "Initializes weights for linear layers using `torch.nn.init.xavier_uniform_`"
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(bias)

In [ ]:
torch.manual_seed(0)
layers = nn.Sequential(
    nn.Linear(2,12),
    nn.ReLU(),
    nn.Linear(12,6),
    nn.ReLU(),
    nn.Linear(6,1)
)
layers.apply(init_xavier_uniform_weights)

Sequential(
  (0): Linear(in_features=2, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=6, bias=True)
  (3): ReLU()
  (4): Linear(in_features=6, out_features=1, bias=True)
)

In [ ]:
#|export
def init_uniform_weights(m:Module,bound):
    "Initializes weights for linear layers using `torch.nn.init.uniform_`"
    if type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight,-bound,bound)

In [ ]:
#|export
def init_kaiming_normal_weights(m:Module,bias=0.01):
    "Initializes weights for linear layers using `torch.nn.init.kaiming_normal_`"
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(bias)

In [ ]:
#|export
def ddpg_conv2d_block(
        # A tuple of state sizes generally representing an image of format: 
        # [channel,width,height]
        state_sz:Tuple[int,int,int],
        # Number of filters to use for each conv layer
        filters=32,
        # Activation function between each layer.
        activation_fn=nn.ReLU,
        # We assume the channels dim should be size 3 max. If it is more
        # we assume the width/height are in the location of channel and need to
        # be transposed.
        ignore_warning:bool=False
    ) -> Tuple[nn.Sequential,int]: # (Convolutional block,n_features_out)
    "Creates a 3 layer conv block from `state_sz` along with expected n_feature output shape."
    channels = state_sz[0]
    if channels>3 and not ignore_warning:
        warn(f'Channels is {channels}>3 in state_sz {state_sz}')
    layers = nn.Sequential(
        nn.BatchNorm2d(channels),
        nn.Conv2d(channels,channels,filters),
        activation_fn(),
        nn.Conv2d(channels,channels,filters),
        activation_fn(),
        nn.Conv2d(channels,channels,filters),   
        nn.Flatten()
    )
    m_layers = deepcopy(layers).to(device='meta')
    out_sz = m_layers(torch.ones((1,*state_sz),device='meta')).shape[-1]
    return layers.to(device='cpu'),out_sz


In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()